# **Projeto Integrado Feature Engineering**





In [1]:
# Importação das bibliotecas necessárias para manipulação de dados, visualização e técnicas de machine learning.
import numpy as np
import pandas as pd

In [2]:
# Carregando o conjunto de dados que será utilizado para o projeto.

df_descritivo = pd.read_excel(r"descritivo_características.xlsx",engine='openpyxl')
dataset = pd.read_csv("./dataset.txt", sep="\t", encoding="latin1")
dataset.columns = df_descritivo['Característica'].tolist()

C:\Users\guilherme.luz\AppData\Local\Temp\ipykernel_17528\2983171997.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("./dataset.txt", sep="\t", encoding="latin1")


## **Modelo Baseline**

In [3]:
#Importando bibliotecas necessárias para o modelo

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
# Criar um novo dataset apenas com as colunas que não possuem nenhum valor nulo
dataset_sem_nulos = dataset.dropna(axis=1, how='any')

In [5]:
# Separando o dataset entre treino e teste
# A critério de um baseline, não iremos utilizar as variáveis categóricas

X = dataset_sem_nulos.drop(columns=['ROTULO_ALVO_MAU=1'])
X = X.select_dtypes(include=['int64', 'float64'])
y = dataset_sem_nulos['ROTULO_ALVO_MAU=1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Normalizando as variaveis numéricas

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Define the KNN Classifier
knn = KNeighborsClassifier()

# Define o range de K
k_values = range(1, 50)

# Criação das listas para armazenar o resultado
accuracies = []
reports = []
matrices = []

# Loop para cada valor de K value e avaliação do modelo
for k in k_values:
    knn.set_params(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    # Cálculo de acurácia
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    # Geração do reporte de classificação
    report = classification_report(y_test, y_pred)
    reports.append(report)
    
    # Geração da matriz de confusão
    matrix = confusion_matrix(y_test, y_pred)
    matrices.append(matrix)

# Buscando o K com maior acurácia
best_k = k_values[accuracies.index(max(accuracies))]

print(f"Melhor K value: {best_k}")
print(f"Acurácia: {max(accuracies)}")
print(reports[accuracies.index(max(accuracies))])
print(matrices[accuracies.index(max(accuracies))])

Melhor K value: 46
Acurácia: 0.7418
              precision    recall  f1-score   support

           0       0.74      1.00      0.85     11132
           1       0.43      0.00      0.01      3868

    accuracy                           0.74     15000
   macro avg       0.59      0.50      0.43     15000
weighted avg       0.66      0.74      0.63     15000

[[11112    20]
 [ 3853    15]]


O modelo sem dados nulos apresentou 74% de assertividade, porém uma grande dificuldade de mapear maus credores, com assertividade de apenas 43%.

_Avaliação das variáveis que mais impactaram o baseline_

In [8]:
# from sklearn.inspection import permutation_importance
# import matplotlib.pyplot as plt

# # Treino do KNN com o melhor valor de k
# knn = KNeighborsClassifier(n_neighbors=best_k)
# knn.fit(X_train, y_train)

# # Calculo de Permutation Importance para avaliar qual variável teve o maior impacto no modelo

# results = permutation_importance(knn, X_test, y_test, n_repeats=10, random_state=42)

# # Feature Importance Scores
# importances = results.importances_mean

# # Plot de scores
# plt.barh(X.columns, importances)
# plt.xlabel('Permutation Importance')
# plt.ylabel('Feature')
# plt.title('Permutation Feature Importance')
# plt.show()

## **Data Understanding & Cleaning**

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 54 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID_CLIENTE                         49999 non-null  int64  
 1   TIPO_FUNCIONARIO                   49999 non-null  object 
 2   DIA_PAGAMENTO                      49999 non-null  int64  
 3   TIPO_ENVIO_APLICACAO               49999 non-null  object 
 4   QUANT_CARTOES_ADICIONAIS           49999 non-null  int64  
 5   TIPO_ENDERECO_POSTAL               49999 non-null  int64  
 6   SEXO                               49999 non-null  object 
 7   ESTADO_CIVIL                       49999 non-null  int64  
 8   QUANT_DEPENDENTES                  49999 non-null  int64  
 9   NIVEL_EDUCACIONAL                  49999 non-null  int64  
 10  ESTADO_NASCIMENTO                  49999 non-null  object 
 11  CIDADE_NASCIMENTO                  49999 non-null  obj

In [10]:
dataset.head()

,ID_CLIENTE,TIPO_FUNCIONARIO,DIA_PAGAMENTO,TIPO_ENVIO_APLICACAO,QUANT_CARTOES_ADICIONAIS,TIPO_ENDERECO_POSTAL,SEXO,ESTADO_CIVIL,QUANT_DEPENDENTES,NIVEL_EDUCACIONAL,...,FLAG_DOCUMENTO_RESIDENCIAL,FLAG_RG,FLAG_CPF,FLAG_COMPROVANTE_RENDA,PRODUTO,FLAG_REGISTRO_ACSP,IDADE,CEP_RESIDENCIAL_3,CEP_PROFISSIONAL_3,ROTULO_ALVO_MAU=1
0,2,C,15,Carga,0,1,F,2,0,0,...,0,0,0,0,1,N,34,230,230,1
1,3,C,5,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,27,591,591,0
2,4,C,20,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,61,545,545,0
3,5,C,10,Web,0,1,M,2,0,0,...,0,0,0,0,1,N,48,235,235,1
4,6,C,10,0,0,1,M,2,0,0,...,0,0,0,0,2,N,40,371,371,1


In [11]:
dataset.describe()

,ID_CLIENTE,DIA_PAGAMENTO,QUANT_CARTOES_ADICIONAIS,TIPO_ENDERECO_POSTAL,ESTADO_CIVIL,QUANT_DEPENDENTES,NIVEL_EDUCACIONAL,NACIONALIDADE,TIPO_RESIDENCIA,MESES_RESIDENCIA,...,TIPO_OCUPACAO,CODIGO_PROFISSAO_CONJUGE,NIVEL_EDUCACIONAL_CONJUGE,FLAG_DOCUMENTO_RESIDENCIAL,FLAG_RG,FLAG_CPF,FLAG_COMPROVANTE_RENDA,PRODUTO,IDADE,ROTULO_ALVO_MAU=1
count,49999.000000,49999.000000,49999.0,49999.000000,49999.000000,49999.000000,49999.0,49999.000000,48650.000000,46222.000000,...,42686.000000,21116.000000,17662.000000,49999.0,49999.0,49999.0,49999.0,49999.000000,49999.000000,49999.000000
mean,25001.000000,12.870077,0.0,1.006540,2.148323,0.650513,0.0,0.961599,1.252230,9.727035,...,2.484281,3.797926,0.296003,0.0,0.0,0.0,0.0,1.275706,43.248745,0.260805
std,14433.612391,6.608357,0.0,0.080607,1.322751,1.193666,0.0,0.202107,0.867841,10.668928,...,1.532262,5.212168,0.955688,0.0,0.0,0.0,0.0,0.988295,14.989115,0.439078
min,2.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,6.000000,0.000000
25%,12501.500000,10.000000,0.0,1.000000,1.000000,0.000000,0.0,1.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,31.000000,0.000000
50%,25001.000000,10.000000,0.0,1.000000,2.000000,0.000000,0.0,1.000000,1.000000,6.000000,...,2.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,41.000000,0.000000
75%,37500.500000,15.000000,0.0,1.000000,2.000000,1.000000,0.0,1.000000,1.000000,15.000000,...,4.000000,11.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,53.000000,1.000000
max,50000.000000,25.000000,0.0,2.000000,7.000000,53.000000,0.0,2.000000,5.000000,228.000000,...,5.000000,17.000000,5.000000,0.0,0.0,0.0,0.0,7.000000,106.000000,1.000000


A partir da descrição estatística das variáveis, é possível notar que as variáveis quantitativas "QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG", "FLAG_CPF", "FLAG_COMPROVANTE_RENDA" estão com todos seus valores nulos, e, portanto, serão descartadas.
Outras colunas como ID_CLIENTE, não agregam nada ao modelo e também será descartada

Também estamos removendo ESTADO_CIVIL, pois existem 7 valores diferentes para estado civil, quando na realidade não existem mais do que 5 de acordo com a legislação brasileira: Solteiro(a), Casado(a), Separado(a), Divorciado(a), Viúvo(a), além do fato de não haver informação sobre esta codificação disponível no dataset.

Mais para frente, criaremos uma variável booleana chamada TEM_CONJUGE, a partir da presença de valor na coluna CODIGO_PROFISSAO_CONJUGE.

In [12]:
#Retirar as colunas "QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG"
#"FLAG_CPF", "FLAG_COMPROVANTE_RENDA" do dataset

columns_to_drop = ["ID_CLIENTE","QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG", "FLAG_CPF", "FLAG_COMPROVANTE_RENDA", "ESTADO_CIVIL"]
dataset = dataset.drop(columns=columns_to_drop)


In [13]:
# Analisar quantos valores distintos existem para as colunas categoricas

# Selecionar apenas as colunas categóricas
categorical_columns = dataset.select_dtypes(include=['object']).columns

# Iterar sobre as colunas categóricas e imprimir o número de valores distintos
for column in categorical_columns:
  print(f"Coluna: {column}, Número de valores distintos: {dataset[column].nunique()}")


Coluna: TIPO_FUNCIONARIO, Número de valores distintos: 1
Coluna: TIPO_ENVIO_APLICACAO, Número de valores distintos: 3
Coluna: SEXO, Número de valores distintos: 4
Coluna: ESTADO_NASCIMENTO, Número de valores distintos: 29
Coluna: CIDADE_NASCIMENTO, Número de valores distintos: 9910
Coluna: ESTADO_RESIDENCIAL, Número de valores distintos: 27
Coluna: CIDADE_RESIDENCIAL, Número de valores distintos: 3528
Coluna: BAIRRO_RESIDENCIAL, Número de valores distintos: 14511
Coluna: FLAG_TELEFONE_RESIDENCIAL, Número de valores distintos: 2
Coluna: CODIGO_AREA_TELEFONE_RESIDENCIAL, Número de valores distintos: 102
Coluna: FLAG_TELEFONE_MOVEL, Número de valores distintos: 1
Coluna: EMPRESA, Número de valores distintos: 2
Coluna: ESTADO_PROFISSIONAL, Número de valores distintos: 28
Coluna: CIDADE_PROFISSIONAL, Número de valores distintos: 2236
Coluna: BAIRRO_PROFISSIONAL, Número de valores distintos: 5057
Coluna: FLAG_TELEFONE_PROFISSIONAL, Número de valores distintos: 2
Coluna: CODIGO_AREA_TELEFONE_

A partir da contagem de valores distintos das variáveis categoricas, é possível notar que as colunas "TIPO_FUNCIONARIO", "FLAG_TELEFONE_MOVEL", "FLAG_REGISTRO_ACSP" estão com todos seus valores iguais, e, portanto, serão descartadas.

In [14]:
columns_cat_to_drop = ["TIPO_FUNCIONARIO", "FLAG_TELEFONE_MOVEL", "FLAG_REGISTRO_ACSP"]
dataset = dataset.drop(columns=columns_cat_to_drop)

Agora avaliaremos se existem dados vazios, nulos ou NaN nas features 'CEP_PROFISSIONAL_3' e 'CEP_RESIDENCIAL_3', dada a redundância de features relacionadas à localização Residencial e Profissional.

In [15]:
# Check for null, empty, or NaN values in CEP_PROFISSIONAL_3 and CEP_RESIDENCIAL_3
cep_columns = ['CEP_PROFISSIONAL_3', 'CEP_RESIDENCIAL_3']

for column in cep_columns:
    null_count = dataset[column].isnull().sum()
    empty_count = (dataset[column].astype(str).str.strip() == '').sum()
    nan_count = dataset[column].isna().sum()
    
    print(f"\nColumn: {column}")
    print(f"Null values: {null_count}")
    print(f"Empty values: {empty_count}")
    print(f"NaN values: {nan_count}")
    
    if null_count > 0 or empty_count > 0 or nan_count > 0:
        print(f"Total problematic values: {null_count + empty_count + nan_count}")
        print(f"Percentage of problematic values: {((null_count + empty_count + nan_count) / len(dataset)) * 100:.2f}%")
    else:
        print("No problematic values found.")


# Count the number of rows with any value under 'CEP_PROFISSIONAL_3' and 'CEP_RESIDENCIAL_3'
cep_columns = ['CEP_PROFISSIONAL_3', 'CEP_RESIDENCIAL_3']

for column in cep_columns:
    non_empty_count = dataset[column].notna().sum()
    total_rows = len(dataset)
    percentage = (non_empty_count / total_rows) * 100
    
    print(f"\nColumn: {column}")
    print(f"Rows with values: {non_empty_count}")
    print(f"Total rows: {total_rows}")
    print(f"Percentage of rows with values: {percentage:.2f}%")


Column: CEP_PROFISSIONAL_3
Null values: 0
Empty values: 0
NaN values: 0
No problematic values found.

Column: CEP_RESIDENCIAL_3
Null values: 0
Empty values: 0
NaN values: 0
No problematic values found.

Column: CEP_PROFISSIONAL_3
Rows with values: 49999
Total rows: 49999
Percentage of rows with values: 100.00%

Column: CEP_RESIDENCIAL_3
Rows with values: 49999
Total rows: 49999
Percentage of rows with values: 100.00%


In [16]:
# Drop columns related to professional and residential location details
columns_loc_to_drop = [
    'ESTADO_PROFISSIONAL', 'CIDADE_PROFISSIONAL', 'BAIRRO_PROFISSIONAL',
    'ESTADO_RESIDENCIAL', 'CIDADE_RESIDENCIAL', 'BAIRRO_RESIDENCIAL'
]

dataset = dataset.drop(columns=columns_loc_to_drop)



In [17]:
columns_nascimento_e_nacionalidade_to_drop = [
    'ESTADO_NASCIMENTO','CIDADE_NASCIMENTO', 'NACIONALIDADE'
]
dataset = dataset.drop(columns=columns_nascimento_e_nacionalidade_to_drop)

In [18]:
dataset.head()

,DIA_PAGAMENTO,TIPO_ENVIO_APLICACAO,TIPO_ENDERECO_POSTAL,SEXO,QUANT_DEPENDENTES,FLAG_TELEFONE_RESIDENCIAL,CODIGO_AREA_TELEFONE_RESIDENCIAL,TIPO_RESIDENCIA,MESES_RESIDENCIA,FLAG_EMAIL,...,MESES_NO_TRABALHO,CODIGO_PROFISSAO,TIPO_OCUPACAO,CODIGO_PROFISSAO_CONJUGE,NIVEL_EDUCACIONAL_CONJUGE,PRODUTO,IDADE,CEP_RESIDENCIAL_3,CEP_PROFISSIONAL_3,ROTULO_ALVO_MAU=1
0,15,Carga,1,F,0,Y,20,1.0,1.0,1,...,0,11.0,4.0,11.0,NaN,1,34,230,230,1
1,5,Web,1,F,0,Y,105,1.0,NaN,1,...,0,11.0,NaN,NaN,NaN,1,27,591,591,0
2,20,Web,1,F,0,N,,NaN,NaN,1,...,0,NaN,NaN,NaN,NaN,1,61,545,545,0
3,10,Web,1,M,0,Y,20,1.0,12.0,1,...,0,9.0,5.0,NaN,NaN,1,48,235,235,1
4,10,0,1,M,0,Y,33,1.0,4.0,1,...,0,9.0,2.0,0.0,0.0,2,40,371,371,1


**Após a limpeza inicial de dados saimos de 53 variáveis para 34 sem considerar a variável alvo (ROTULO_ALVO_MAU=1). Os próximos passos serão tratar, de forma simples e rápida, os dados nulos e os dados existentes para aplicar um modelo baseline**

### **Análise de Dados Nulos**

In [19]:
# Calcula a quantidade de valores nulos para cada coluna
null_counts = dataset.isnull().sum()

# Filtra apenas as colunas que possuem pelo menos um valor nulo
columns_with_nulls = null_counts[null_counts > 0]

# Cria um novo DataFrame com as colunas que possuem valores nulos
df_nulls = pd.DataFrame({'Quantidade de Nulos': columns_with_nulls})

# Calcula a porcentagem de valores nulos para cada coluna
df_nulls['Porcentagem de Nulos (%)'] = (df_nulls['Quantidade de Nulos'] / len(dataset)) * 100

# Organiza df_nulls em ordem crescente da coluna 'Quantidade de Nulos'
df_nulls = df_nulls.sort_values(by='Quantidade de Nulos')


# Imprime o DataFrame com a quantidade e a porcentagem de valores nulos para cada coluna
print("Colunas com valores nulos:")
print(df_nulls)


Colunas com valores nulos:
                           Quantidade de Nulos  Porcentagem de Nulos (%)
TIPO_RESIDENCIA                           1349                  2.698054
MESES_RESIDENCIA                          3777                  7.554151
TIPO_OCUPACAO                             7313                 14.626293
CODIGO_PROFISSAO                          7756                 15.512310
CODIGO_PROFISSAO_CONJUGE                 28883                 57.767155
NIVEL_EDUCACIONAL_CONJUGE                32337                 64.675294


Nota-se que 6 variáveis apresentam valores nulos

#### Colunas com menos de 10% de valores nulls

A exclusão de linhas com menos de 10% de valores nulos é uma estratégia adequada para um modelo baseline, principalmente porque tem um impacto mínimo no conjunto de dados como um todo. Ao remover apenas uma pequena fração das linhas, preserva-se a grande maioria dos dados, mantendo assim a estrutura geral e as relações entre as variáveis praticamente inalteradas. Esta abordagem oferece uma solução simples e rápida, com baixo risco de introduzir viés significativo ou alterar substancialmente as características do dataset. O impacto reduzido nos dados permite uma análise inicial eficiente, enquanto mantém a integridade e a representatividade do conjunto original, tornando-a uma escolha pragmática para as primeiras etapas do processo de modelagem.

In [20]:
# Imprimindo os diferentes tipos de valores em 'TIPO_RESIDENCIA'
print("Tipos de valores únicos em TIPO_RESIDENCIA:")
print(dataset['TIPO_RESIDENCIA'].unique())
print("\nContagem de cada tipo de valor:")
print(dataset['TIPO_RESIDENCIA'].value_counts(dropna=False))

# Tratamento de valores nulos para TIPO_RESIDENCIA usando a moda
tipo_residencia_mode = dataset['TIPO_RESIDENCIA'].mode()[0]
dataset['TIPO_RESIDENCIA'] = dataset['TIPO_RESIDENCIA'].fillna(tipo_residencia_mode)
print("Tratamento de valores nulos para TIPO_RESIDENCIA concluído.")
print("Moda utilizada para preenchimento:", tipo_residencia_mode)
print("Quantidade de valores nulos restantes em TIPO_RESIDENCIA:", dataset['TIPO_RESIDENCIA'].isnull().sum())

print("Tipos de valores únicos em TIPO_OCUPACAO:")
print(dataset['TIPO_OCUPACAO'].unique())
print("\nContagem de cada tipo de valor:")
print(dataset['TIPO_OCUPACAO'].value_counts(dropna=False))

# Tratamento de valores nulos para TIPO_OCUPACAO usando a moda
tipo_ocupacao_mode = dataset['TIPO_OCUPACAO'].mode()
dataset['TIPO_OCUPACAO'] = dataset['TIPO_OCUPACAO'].fillna(tipo_ocupacao_mode)
print("Tratamento de valores nulos para MESES_RESIDENCIA e TIPO_OCUPACAO concluído.")
print("Mediana utilizada para preenchimento de TIPO_OCUPACAO:", tipo_ocupacao_mode)
print("Quantidade de valores nulos restantes em TIPO_OCUPACAO:", dataset['TIPO_OCUPACAO'].isnull().sum())


Tipos de valores únicos em TIPO_RESIDENCIA:
[ 1. nan  2.  5.  0.  3.  4.]

Contagem de cada tipo de valor:
TIPO_RESIDENCIA
1.0    41571
2.0     3884
5.0     1983
NaN     1349
0.0      760
4.0      311
3.0      141
Name: count, dtype: int64
Tratamento de valores nulos para TIPO_RESIDENCIA concluído.
Moda utilizada para preenchimento: 1.0
Quantidade de valores nulos restantes em TIPO_RESIDENCIA: 0
Tipos de valores únicos em TIPO_OCUPACAO:
[ 4. nan  5.  2.  1.  0.  3.]

Contagem de cada tipo de valor:
TIPO_OCUPACAO
2.0    16947
1.0     8742
NaN     7313
4.0     6999
5.0     6891
0.0     2788
3.0      319
Name: count, dtype: int64
Tratamento de valores nulos para MESES_RESIDENCIA e TIPO_OCUPACAO concluído.
Mediana utilizada para preenchimento de TIPO_OCUPACAO: 0    2.0
Name: TIPO_OCUPACAO, dtype: float64
Quantidade de valores nulos restantes em TIPO_OCUPACAO: 7313


In [21]:
# Tratamento de valores nulos para MESES_RESIDENCIA usando a mediana
meses_residencia_median = dataset['MESES_RESIDENCIA'].median()
dataset['MESES_RESIDENCIA'] = dataset['MESES_RESIDENCIA'].fillna(meses_residencia_median)
print("Tratamento de valores nulos para MESES_RESIDENCIA concluído.")
print("Mediana utilizada para preenchimento de MESES_RESIDENCIA:", meses_residencia_median)
print("Quantidade de valores nulos restantes em MESES_RESIDENCIA:", dataset['MESES_RESIDENCIA'].isnull().sum())


Tratamento de valores nulos para MESES_RESIDENCIA concluído.
Mediana utilizada para preenchimento de MESES_RESIDENCIA: 6.0
Quantidade de valores nulos restantes em MESES_RESIDENCIA: 0


#### Tratamento de Valores Nulos em Variáveis Categóricas

Colunas tratadas:
- TIPO_OCUPACAO 
- CODIGO_PROFISSAO
- CODIGO_PROFISSAO_CONJUGE
- NIVEL_EDUCACIONAL_CONJUGE
- BAIRRO_PROFISSIONAL
- CIDADE_PROFISSIONAL

Justificativa da abordagem:
1. Consistência: Aplicamos o mesmo tratamento para todas as variáveis categóricas com valores ausentes.
2. Preservação da informação: A categoria "Não informado" mantém a distinção entre dados fornecidos e não fornecidos.
3. Interpretabilidade: Facilita a compreensão dos dados ausentes nas análises subsequentes.
4. Adequação ao contexto: Para informações como profissão, nível educacional e localização, "Não informado" é uma representação apropriada da ausência de dados.

In [22]:
# Substituindo valores nulos por 999 para colunas numéricas
colunas_numericas = ['CODIGO_PROFISSAO', 'CODIGO_PROFISSAO_CONJUGE', 'NIVEL_EDUCACIONAL_CONJUGE']
for coluna in colunas_numericas:
    dataset[coluna] = dataset[coluna].fillna(999)




In [23]:
# Calcula a quantidade de valores nulos para cada coluna
null_counts = dataset.isnull().sum()

# Filtra apenas as colunas que possuem pelo menos um valor nulo
columns_with_nulls = null_counts[null_counts > 0]

# Cria um novo DataFrame com as colunas que possuem valores nulos
df_nulls = pd.DataFrame({'Quantidade de Nulos': columns_with_nulls})

# Calcula a porcentagem de valores nulos para cada coluna
df_nulls['Porcentagem de Nulos (%)'] = (df_nulls['Quantidade de Nulos'] / len(dataset)) * 100

# Organiza df_nulls em ordem crescente da coluna 'Quantidade de Nulos'
df_nulls = df_nulls.sort_values(by='Quantidade de Nulos')


# Imprime o DataFrame com a quantidade e a porcentagem de valores nulos para cada coluna
print("Colunas com valores nulos:")
print(df_nulls)

Colunas com valores nulos:
               Quantidade de Nulos  Porcentagem de Nulos (%)
TIPO_OCUPACAO                 7313                 14.626293


### **Análise dos Dados Existentes**

_Para as variáveis onde há todos os dados, é necessário entender se os dados existentes fazem sentido_

Vamos começar analisando as variáveis binárias e categoricas. Variáveis como renda e patrimônio possuem muitos valores possíveis e sua coerência será analisada posteriormente a partir de análises de outliers. Enquanto as variáveis de texto como cidade e bairro não serão analisadas por existir inumeras possibilidades. As variáveis e CEP serão analisadas isoladamente

#### **Análise Binárias e Categoricas**

_Aqui buscamos entender os tipos de dados únicos existentes para variáveis binárias e categóricas_

In [24]:
columns_to_drop_x = ["CIDADE_NASCIMENTO", "CIDADE_RESIDENCIAL", "BAIRRO_RESIDENCIAL", "RENDA_PESSOAL_MENSAL", "OUTRAS_RENDAS", "VALOR_PATRIMONIO_PESSOAL", "CEP_RESIDENCIAL_3", "CEP_PROFISSIONAL_3"]
columns_without_nulls = dataset_sem_nulos.drop(columns=columns_to_drop_x)

In [25]:
# Iterar sobre as colunas e imprimir os valores distintos
for column in columns_without_nulls.columns:
    print(f"Coluna: {column}")
    print(columns_without_nulls[column].unique())
    print("-" * 20)

Coluna: ID_CLIENTE
[    2     3     4 ... 49998 49999 50000]
--------------------
Coluna: TIPO_FUNCIONARIO
['C']
--------------------
Coluna: DIA_PAGAMENTO
[15  5 20 10 25  1]
--------------------
Coluna: TIPO_ENVIO_APLICACAO
['Carga' 'Web' '0']
--------------------
Coluna: QUANT_CARTOES_ADICIONAIS
[0]
--------------------
Coluna: TIPO_ENDERECO_POSTAL
[1 2]
--------------------
Coluna: SEXO
['F' 'M' 'N' ' ']
--------------------
Coluna: ESTADO_CIVIL
[2 1 5 3 7 4 6 0]
--------------------
Coluna: QUANT_DEPENDENTES
[ 0  2  1  3  4  5  6  7 10  8 13 11  9 12 14 15 53]
--------------------
Coluna: NIVEL_EDUCACIONAL
[0]
--------------------
Coluna: ESTADO_NASCIMENTO
['RJ' 'RN' 'PE' 'MG' 'BA' 'SP' 'RS' 'CE' 'PA' 'PB' 'MA' ' ' 'GO' 'AC' 'MT'
 'AL' 'AP' 'TO' 'SC' 'PR' 'MS' 'DF' 'PI' 'RO' 'ES' 'AM' 'SE' 'RR' 'XX']
--------------------
Coluna: NACIONALIDADE
[1 0 2]
--------------------
Coluna: ESTADO_RESIDENCIAL
['RJ' 'RN' 'PE' 'MG' 'BA' 'SP' 'RS' 'CE' 'AP' 'MS' 'DF' 'PB' 'MA' 'PA'
 'GO' 'PR' 'M

### Organizando os tipos de dados


In [26]:
dataset.columns.tolist()

['DIA_PAGAMENTO',
 'TIPO_ENVIO_APLICACAO',
 'TIPO_ENDERECO_POSTAL',
 'SEXO',
 'QUANT_DEPENDENTES',
 'FLAG_TELEFONE_RESIDENCIAL',
 'CODIGO_AREA_TELEFONE_RESIDENCIAL',
 'TIPO_RESIDENCIA',
 'MESES_RESIDENCIA',
 'FLAG_EMAIL',
 'RENDA_PESSOAL_MENSAL',
 'OUTRAS_RENDAS',
 'FLAG_VISA',
 'FLAG_MASTERCARD',
 'FLAG_DINERS',
 'FLAG_AMERICAN_EXPRESS',
 'FLAG_OUTROS_CARTOES',
 'QUANT_CONTAS_BANCARIAS',
 'QUANT_CONTAS_BANCARIAS_ESPECIAIS',
 'VALOR_PATRIMONIO_PESSOAL',
 'QUANT_CARROS',
 'EMPRESA',
 'FLAG_TELEFONE_PROFISSIONAL',
 'CODIGO_AREA_TELEFONE_PROFISSIONAL',
 'MESES_NO_TRABALHO',
 'CODIGO_PROFISSAO',
 'TIPO_OCUPACAO',
 'CODIGO_PROFISSAO_CONJUGE',
 'NIVEL_EDUCACIONAL_CONJUGE',
 'PRODUTO',
 'IDADE',
 'CEP_RESIDENCIAL_3',
 'CEP_PROFISSIONAL_3',
 'ROTULO_ALVO_MAU=1']

In [38]:
datatypes = {
    'DIA_PAGAMENTO': object,
    'TIPO_ENVIO_APLICACAO': object, 
    'TIPO_ENDERECO_POSTAL': object,
    'SEXO': object,
    'QUANT_DEPENDENTES': int,
    'FLAG_TELEFONE_RESIDENCIAL': bool,
    'CODIGO_AREA_TELEFONE_RESIDENCIAL': object,
    'TIPO_RESIDENCIA': object,
    'MESES_RESIDENCIA': int,
    'FLAG_EMAIL': bool,
    'RENDA_PESSOAL_MENSAL': float,
    'OUTRAS_RENDAS': float,
    'FLAG_VISA': bool,
    'FLAG_MASTERCARD': bool,
    'FLAG_DINERS': bool,
    'FLAG_AMERICAN_EXPRESS': bool,
    'FLAG_OUTROS_CARTOES': bool,
    'QUANT_CONTAS_BANCARIAS': int,
    'QUANT_CONTAS_BANCARIAS_ESPECIAIS': int,
    'VALOR_PATRIMONIO_PESSOAL': float,
    'QUANT_CARROS': int,
    'EMPRESA': bool,
    'FLAG_TELEFONE_PROFISSIONAL': bool,
    'CODIGO_AREA_TELEFONE_PROFISSIONAL': object,
    'MESES_NO_TRABALHO': int,
    'CODIGO_PROFISSAO': object,
    'TIPO_OCUPACAO': object,
    'CODIGO_PROFISSAO_CONJUGE': object,
    'NIVEL_EDUCACIONAL_CONJUGE': object,
    'PRODUTO': object,
    'IDADE': int,
    'CEP_RESIDENCIAL_3': str,
    'CEP_PROFISSIONAL_3': str,
    'ROTULO_ALVO_MAU=1': bool
}

In [39]:
dataset = dataset.astype(datatypes)


#### **Análise dados numéricos**

In [40]:
dataset.dtypes

DIA_PAGAMENTO                         object
TIPO_ENVIO_APLICACAO                  object
TIPO_ENDERECO_POSTAL                  object
SEXO                                  object
QUANT_DEPENDENTES                      int32
FLAG_TELEFONE_RESIDENCIAL               bool
CODIGO_AREA_TELEFONE_RESIDENCIAL      object
TIPO_RESIDENCIA                       object
MESES_RESIDENCIA                       int32
FLAG_EMAIL                              bool
RENDA_PESSOAL_MENSAL                 float64
OUTRAS_RENDAS                        float64
FLAG_VISA                               bool
FLAG_MASTERCARD                         bool
FLAG_DINERS                             bool
FLAG_AMERICAN_EXPRESS                   bool
FLAG_OUTROS_CARTOES                     bool
QUANT_CONTAS_BANCARIAS                 int32
QUANT_CONTAS_BANCARIAS_ESPECIAIS       int32
VALOR_PATRIMONIO_PESSOAL             float64
QUANT_CARROS                           int32
EMPRESA                                 bool
FLAG_TELEF

Avaliar quantidade de outliers e distrivuição para tomada de decisão de tratamento e tipo de escalonamento

In [41]:
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

num_dataset = dataset.select_dtypes(exclude='object')

z_scores = zscore(num_dataset)

outlier_counts = (np.abs(z_scores) > 3).sum(axis=0)

print(outlier_counts)

# A quantidade de outliers nas variáveis principais não é algo que necessite de remoção,
# isso pode ser resolvido com o escalonamento

TypeError: loop of ufunc does not support argument 0 of type float which has no callable sqrt method

_Inputando valroes medianos para variáveis nulas_

In [32]:
from sklearn.impute import SimpleImputer

# create an Imputer object with the strategy 'median'
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# fit the imputer to the dataset
imputer = imputer.fit(num_dataset)

# transform the dataset to fill missing values
dataset_filled = imputer.transform(num_dataset)

# create a new dataframe with the filled values
dataset_filled = pd.DataFrame(dataset_filled, columns=num_dataset.columns)

_Escalonamento das variáveis numéricas_

In [33]:
from sklearn.preprocessing import StandardScaler

# assume dataset is your dataframe

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to the dataset
scaler.fit(dataset_filled)

# transform the dataset to scale the data
dataset_scaled = scaler.transform(dataset_filled)

# create a new dataframe with the scaled values
dataset_scaled = pd.DataFrame(dataset_scaled, columns=dataset_filled.columns)

_Criando o dataframe final para inclusão no modelo_

In [44]:
# Criando dummies
# Concatenando dataframe final

cat_columns = dataset.select_dtypes(include='object')

# Perform one-hot encoding on categorical columns
dummies = pd.get_dummies(dataset[cat_columns.columns], drop_first=True, prefix=cat_columns.columns)


final_dataset = pd.concat([dummies,dataset_scaled,dataset['ROTULO_ALVO_MAU=1']],axis=1)

final_dataset.reset_index(drop=True)

final_dataset.head()

,DIA_PAGAMENTO_5,DIA_PAGAMENTO_10,DIA_PAGAMENTO_15,DIA_PAGAMENTO_20,DIA_PAGAMENTO_25,TIPO_ENVIO_APLICACAO_Carga,TIPO_ENVIO_APLICACAO_Web,TIPO_ENDERECO_POSTAL_2,SEXO_F,SEXO_M,...,MESES_RESIDENCIA,RENDA_PESSOAL_MENSAL,OUTRAS_RENDAS,QUANT_CONTAS_BANCARIAS,QUANT_CONTAS_BANCARIAS_ESPECIAIS,VALOR_PATRIMONIO_PESSOAL,QUANT_CARROS,MESES_NO_TRABALHO,IDADE,ROTULO_ALVO_MAU=1
0,False,False,True,False,False,True,False,False,True,False,...,-0.819544,-0.017418,-0.039747,-0.745592,-0.745592,-0.054828,-0.711588,-0.024306,-0.617037,True
1,True,False,False,False,False,False,True,False,True,False,...,-0.334348,-0.049277,-0.039747,-0.745592,-0.745592,-0.054828,-0.711588,-0.024306,-1.084047,False
2,False,False,False,True,False,False,True,False,True,False,...,-0.334348,-0.049277,-0.039747,-0.745592,-0.745592,-0.054828,-0.711588,-0.024306,1.184288,False
3,False,True,False,False,False,False,True,False,False,True,...,0.247888,0.039929,-0.039747,-0.745592,-0.745592,-0.054828,-0.711588,-0.024306,0.316984,True
4,False,True,False,False,False,False,False,False,False,True,...,-0.528426,-0.049277,-0.039747,1.337956,1.337956,-0.054828,1.405308,-0.024306,-0.216742,True


### Modelo final

In [45]:
# Separando o dataset entre treino e teste
# A critério de um baseline, não iremos utilizar as variáveis categóricas

X = final_dataset.drop(columns=['ROTULO_ALVO_MAU=1'])

y = final_dataset['ROTULO_ALVO_MAU=1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [46]:
# Define the KNN Classifier
knn = KNeighborsClassifier()

# Define o range de K
k_values = range(1, 50)

# Criação das listas para armazenar o resultado
accuracies = []
reports = []
matrices = []

# Loop para cada valor de K value e avaliação do modelo
for k in k_values:
    knn.set_params(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    # Cálculo de acurácia
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    # Geração do reporte de classificação
    report = classification_report(y_test, y_pred)
    reports.append(report)
    
    # Geração da matriz de confusão
    matrix = confusion_matrix(y_test, y_pred)
    matrices.append(matrix)

# Buscando o K com maior acurácia
best_k = k_values[accuracies.index(max(accuracies))]

print(f"Melhor K value: {best_k}")
print(f"Acurácia: {max(accuracies)}")
print(reports[accuracies.index(max(accuracies))])
print(matrices[accuracies.index(max(accuracies))])

Melhor K value: 32
Acurácia: 0.7422666666666666
              precision    recall  f1-score   support

       False       0.74      1.00      0.85     11132
        True       0.51      0.01      0.03      3868

    accuracy                           0.74     15000
   macro avg       0.63      0.50      0.44     15000
weighted avg       0.68      0.74      0.64     15000

[[11084    48]
 [ 3818    50]]


In [47]:
# Treino do KNN com o melhor valor de k
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=32)